<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Langchain_Knowledge_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain --progress-bar off
!pip install -qU langchainhub --progress-bar off
!pip install -qU openai --progress-bar off

In [2]:
# @title Connect to google drive
from google.colab import drive
import os
import json

drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

Mounted at /content/drive


In [28]:
# @title Load llm
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="mistralai/mistral-7b-instruct:free",
  openai_api_key=env['openrouter.ai']['apiKey'],
  openai_api_base="https://openrouter.ai/api/v1",
  temperature= 0, max_tokens= 4096
)

# Graph creation

In [62]:
from langchain.prompts import PromptTemplate
from langchain.graphs.networkx_graph import KG_TRIPLE_DELIMITER
from langchain.indexes.prompts.knowledge_triplet_extraction import (
    KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT,
)
from langchain.chains import LLMChain

# We need to customize because of the model we are going to use.
# Prompt template for knowledge triple extraction
_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE = (
    "<s>[INST]You are a networked intelligence helping a human track knowledge triples"
    " about all relevant people, things, concepts, etc. and integrating"
    " them with your knowledge stored within your weights"
    " as well as that stored in a knowledge graph."
    " Extract all of the knowledge triples base on user wrapped in ```."
    " A knowledge triple is a clause that contains a subject, a predicate,"
    " and an object. The subject is the entity being described,"
    " the predicate is the property of the subject that is being"
    " described, and the object is the value of the property.\n\n"
    " If you do your best work I'll tip you $10,000!\n\n"
    "EXAMPLE\n"
    "User: Nevada is a state in the US. It's also the number 1 producer of gold in the US.\n"
    f"Output: (Nevada, is a, state){KG_TRIPLE_DELIMITER}(Nevada, is in, US)"
    f"{KG_TRIPLE_DELIMITER}(Nevada, is the number 1 producer of, gold)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "User: Descartes likes to drive antique scooters and play the mandolin.\n"
    f"Output: (Descartes, likes to drive, antique scooters){KG_TRIPLE_DELIMITER}(Descartes, plays, mandolin)\n"
    "END OF EXAMPLE\n\n"
    "User: ```{input}```\n"
    "Output: [/INST]"
)

KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT = PromptTemplate(
    input_variables=["input"],
    template=_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE,
)

# Create an LLMChain using the knowledge triple extraction prompt
chain = LLMChain(llm=llm, prompt=KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT)

In [65]:
# Run the chain with the specified text
text = "The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris."
triples = chain.invoke(
    {'input' : text}
)
triples

{'input': 'The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris.',
 'text': '(Paris, is the capital of, France)\n<|>(Paris, is the most populous city of, France)\n<|>(Eiffel Tower, is a famous landmark in, Paris)'}

In [66]:
from pprint import pprint

def parse_triples(response, delimiter=KG_TRIPLE_DELIMITER):
    if not response:
        return []
    return response.split(delimiter)

triples_list = parse_triples(triples.get("text"))

pprint(triples_list)

['(Paris, is the capital of, France)\n',
 '(Paris, is the most populous city of, France)\n',
 '(Eiffel Tower, is a famous landmark in, Paris)']


In [ ]:
# @title Sample document

DOCUMENT = """
When the Talent does have a Portuguese company contract
Check what is the Nationality of the Talent?
If the talent is a Nationality of EU or FRA, Talent can travel to portugal and start working and the  needs to submit the following documents
                               		Valid ID document, Proof of address (lease agreement) and Work contract
                                  If the talent is not a Nationality of EU or FRA, check where the talent is residing ?
 If the talent is legally residing in EU and not in Portugal, check the talent permit type
 If the talent holds a blue card permit less than 18 months,
                    Talent can wait to complete 18 months or start the visa process.
If the talent holds a blue card permit greater than 18 months, Talent can travel to portugal and start working and the talent needs to submit the following documents
                               Valid ID document,
      Proof of address (lease agreement) and
      Work contract
If the  talent holds a regular permit then the stay in Portugal needs to be checked.
If the talent is legally residing in EU and in Portugal, the talent might qualify for a employment permit
If the talent is not legally residing in the EU, check if the stay in Portugal is longer than 12 months . If longer than 12 months, check the talent employment type. If the employment type is a contractor, talent may qualify for a self employment visa.  If the employment type is new employment, then check if the academic background is related to work in Portugal and if it is related, check the years of experience.  If the experience is less than 4, the talent may qualify for a regular employment visa.  If the experience is greater than 4 , check the talent monthly salary. If monthly salary is above 1500, the talent may qualify for a Highly qualified employment Visa.  If it is less than 1500, the talent may qualify for a regular employment visa
On the other hand if the  employment type is new employment, and when the academic background is not related to work in Portugal, the talent may qualify for a regular employment visa
If the talent is not legally residing in the EU and if the stay in Portugal is shorter than 12 months, Talent might qualify for a temporary stay visa



When the Talent does not have a Portuguese company contract
Check the nationality of the Talent.
If the talent Nationality is EU, the talent can travel to PT and start working as an independent worker.
If the talent Nationality is NON EU, check where is the talent legally residing ?
If the talent is legally residing in Portugal, the talent might qualify for a self employment permit.
If the talent is legally residing in other countries, check if the stay in Portugal is longer than 12 months.
If the stay is longer, check the talent’s employment type .
If it is New employment, then please contact us for further analysis of your options.
The talent's employment type is an extension of employment, check how much time has the talent been employed by the sending company?
                                                 - If it is 12 months or longer, Talent might qualify for an ICT
                                                 -  If it is shorter than 12 months, check the talent’s monthly salary
 If salary greater than 3040 Talent may qualify for a digital nomad visa .
If salary is less than 3040, then please contact us for further analysis of your options.
The talent’s employment type is contractor, check the talent’s monthly salary
 If salary greater than 3040 Talent may qualify for a digital nomad visa .
If salary is less than 3040, then please contact us for further analysis of your options.
"""